##  Importa las distintas librerias

In [ ]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D
from keras import backend as K

## Limpia toda la informacion de Keras

In [ ]:
K.clear_session()

## Importar imagenes

In [ ]:
images_entrenamiento = './caras/entrenamiento'
images_validacion = './caras/validacion'

## Ingreso parametros
### Modificables en el tiempo

##### valor lr es altamente modificable 
##### filtros agregar mas?
##### hacer vs segun capas?

In [ ]:
epocas = 20
longitud = 150
altura = 150
batch_size = 50
steps_per_epoch = 1000
validation_steps = 300
filterConv1 = 64
filterConv2 = 32
size_filter1 = (3, 3)
size_filter2 = (2, 2)
size_pool = (2, 2)
class_model = 8
lr = 0.0001

## Preparacion de las imagenes para ser procesadas

In [ ]:
entrenamiento_data_generador = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_data_generador = ImageDataGenerator(
    rescale = 1. / 255
)

entrenamiento_generador = entrenamiento_data_generador.flow_from_directory(
    images_entrenamiento,
    target_size = (altura, longitud),
    batch_size = batch_size
)

validacion_generador = test_data_generador.flow_from_directory(
    images_validacion,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical'
)

## Comienza modelo

#### Modelo puede sufrir cambios, los cuales pueden ser principalmente valor de filtro que se cambia en declaracion de variables

In [ ]:
cnn = Sequential()

cnn.add(Convolution2D(
    filterConv1,
    size_filter1,
    padding = "same",
    input_shape = (longitud, altura, 3),
    activation = 'relu'
))
cnn.add(MaxPooling2D(
    pool_size = size_pool
))

cnn.add(Convolution2D(
    filterConv2,
    size_filter2,
    padding = "same"
))
cnn.add(MaxPooling2D(
    pool_size = size_pool
))

cnn.add(Flatten())

cnn.add(Dense(
    256,
    activation = "relu"
))

cnn.add(Dropout(
    0.5
))

cnn.add(Dense(
    class_model,
    activation = "softmax"
))

cnn.compile(
    loss = "categorical_crossentropy",
    optimizer = optimizers.Adam(lr = lr),
    metrics = ['accuracy']
)

cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch = steps_per_epoch,
    epochs = epocas,
    validation_data = validacion_generador,
    validation_steps = validation_steps
)

## Guardar modelo

In [ ]:
target_dir = "./modelo/"
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

cnn.save("./modelo/modelo.h5")
cnn.save_weights("./modelo/peso.h5")